This tutorial will demonstrate how to pre-process single-cell raw UMI counts to generate expression matrices that can be used as input to cell-cell communication tools. We will assume appropriate quality-control (QC) has already been applied to the dataset (e.g., exclusion of low-quality cells and doublets). We recommend the tutorial by [Luecken & Theis](https://doi.org/10.15252/msb.20188746) as a starting point for a detailed overview of QC and single-cell RNAseq analysis pipelines in general. 

Here we will focus on:
1. Normalization
2. Batch correction (for multiple samples/contexts)

We demonstrate a typical workflow using the popular single-cell analysis [Seurat](https://satijalab.org/seurat/index.html). Preferably, the workflow can maintain non-negative counts since Tensor-cell2cell uses a non-negative decomposition.

For use with Tensor-cell2cell, we want a dataset that represents >2 contexts. We also want a dataset that contains [replicates](https://www.nature.com/articles/nmeth.3091). Replicates will allow us to ensure that the output factors are not simply due to technical effects (i.e., a factor with high loadings for just one replicate in the context dimension). We will use a [BALF COVID dataset](https://doi.org/10.1038/s41591-020-0901-9), which contains 12 samples associated with "Healthy Control", "Moderate", or "Severe" COVID contexts. This dataset does not contain technical replicates since each sample was taken from a different patient, but each sample associated with a context is a biological replicate. [Batch correction](https://www.nature.com/articles/s41592-018-0254-1) removes technical variation while preserving biological variation bewteen samples. We can reasonably assume that the biological variation in samples between contexts will be greater than that of those within contexts after using appropriate batch correction to remove technical variation. Thus, we expect Tensor-cell2cell to capture overall communication trends differing between contexts and can then assess that output factors aren't simply due to technical effects by checking that the output factors have similar loadings for biological replicates and do not have  high loadings for just one sample in the context dimension. 

In [1]:
library(Seurat)

Attaching SeuratObject



The 12 samples can be downloaded as .h5 files from [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE145926). You can also download the cell metadata from [here](https://raw.githubusercontent.com/zhangzlab/covid_balf/master/all.cell.annotation.meta.txt)

We download these files directly in the proceeding cell:

In [4]:
data.path<-'/data3/hratch/ccc_protocols/raw/covid_balf/'

# download the metadata
metadata.link <- 'https://raw.githubusercontent.com/zhangzlab/covid_balf/master/all.cell.annotation.meta.txt'
cmd <- paste0('wget ', metadata.link, ' -O ', data.path, 'metadata.txt')
system(cmd, ignore.stdout = T, ignore.stderr = T)

# download the expression data
sample.links <- c(
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339769/suppl/GSM4339769%5FC141%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5',
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339770/suppl/GSM4339770%5FC142%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5',
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339771/suppl/GSM4339771%5FC143%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339772/suppl/GSM4339772%5FC144%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339773/suppl/GSM4339773%5FC145%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5',
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4339nnn/GSM4339774/suppl/GSM4339774%5FC146%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475048/suppl/GSM4475048%5FC51%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475049/suppl/GSM4475049%5FC52%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475050/suppl/GSM4475050%5FC100%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475051/suppl/GSM4475051%5FC148%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5', 
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475052/suppl/GSM4475052%5FC149%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5',
    'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4475nnn/GSM4475053/suppl/GSM4475053%5FC152%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5'
    )

for (sl in sample.links){
    cmd <- paste0('wget ', sl, ' -P ', data.path)
    system(cmd, ignore.stdout = T, ignore.stderr = T)
}

We can then format the downloaded files:

In [23]:
# format the metadata
md <- read.table(paste0(data.path, 'metadata.txt'), header = T, row.names = 'ID')
colnames(md) = c('Sample.ID', 'sample_new', 'Context', 'disease', 'hasnCoV', 'cluster', 'cell.type')

context.map = c('Healthy.Control', 'Moderate.Covid', 'Severe.Covid')
names(context.map) <- c('HC', 'M', 'S')
md['Context'] <- unname(context.map[md$Context])
md$Context <- factor(md$Context, levels = c('Healthy.Control', 'Moderate.Covid', 'Severe.Covid'))

md<-md[md$Sample.ID != 'GSM3660650', ] # drop the non-scRNAseq dataset included in this file

md<-md[with(md, order(Context, Sample.ID)), ]
head(md)

,Sample.ID,sample_new,Context,disease,hasnCoV,cluster,cell.type
,<chr>,<chr>,<fct>,<chr>,<chr>,<int>,<chr>
AAACCCACAGCTACAT_3,C100,HC3,Healthy.Control,N,N,27,B
AAACCCATCCACGGGT_3,C100,HC3,Healthy.Control,N,N,23,Macrophages
AAACCCATCCCATTCG_3,C100,HC3,Healthy.Control,N,N,6,T
AAACGAACAAACAGGC_3,C100,HC3,Healthy.Control,N,N,10,Macrophages
AAACGAAGTCGCACAC_3,C100,HC3,Healthy.Control,N,N,10,Macrophages
AAACGAAGTCTATGAC_3,C100,HC3,Healthy.Control,N,N,9,T


In [20]:
balf.samples<-list()

suppressMessages({
    suppressWarnings({
        for (filename in list.files(data.path)){
            if (endsWith(filename, '.h5')){
                sample<-unlist(strsplit(filename, '_'))[[2]]

                # subset and format metadata
                md.sample<-md[md$Sample.ID == sample,]
                rownames(md.sample) <- unname(sapply(rownames(md.sample), 
                                                   function(x) paste0(unlist(strsplit(x, '_'))[[1]], '-1')))
                # load the counts
                so <- Seurat::Read10X_h5(filename=paste0(data.path, filename), unique.features=T)
                so <- so[, rownames(md.sample)] # only include cells present in the metadata

                # preprocess
                so <- CreateSeuratObject(counts=so, project=sample, meta.data=md.sample[c('Sample.ID', 'Context', 'cell.type')], 
                                          min.cells=3)        
                balf.samples[[sample]] <- so
            }
        }        
    })
})

balf.samples is a list with names as each sample and values as a Seurat object storing the raw UMI counts for that sample

In [25]:
names(balf.samples)

[1] "C141" "C142" "C143" "C144" "C145" "C146" "C51"  "C52"  "C100" "C148"
[11] "C149" "C152"

In [28]:
balf.samples$C100

An object of class Seurat 
16566 features across 2566 samples within 1 assay 
Active assay: RNA (16566 features, 0 variable features)

To normalize the raw UMI counts, we recommend log(1+CPM) normalization, as this maintains non-negative counts and is the input for many communication scoring functions

In [29]:
balf.samples <- lapply(balf.samples, 
                    function(so) NormalizeData(so, normalization.method = "LogNormalize", scale.factor = 1e6))

In [36]:
head(as.data.frame(balf.samples[['C100']]@assays$RNA@data))

,AAACCCACAGCTACAT-1,AAACCCATCCACGGGT-1,AAACCCATCCCATTCG-1,AAACGAACAAACAGGC-1,AAACGAAGTCGCACAC-1,AAACGAAGTCTATGAC-1,AAACGAAGTGTAGTGG-1,AAACGCTGTCACGTGC-1,AAACGCTGTTGGAGGT-1,AAAGAACTCTAGAACC-1,⋯,TTTGATCTCCCGAAAT-1,TTTGGAGCAATACAGA-1,TTTGGAGTCACCATAG-1,TTTGGAGTCTCACCCA-1,TTTGGTTAGATGGCGT-1,TTTGGTTGTACCCAGC-1,TTTGGTTGTTACTCAG-1,TTTGTTGAGCTAGAGC-1,TTTGTTGCAATGAAAC-1,TTTGTTGCAGAGGGTT-1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AL627309.1,0,0,0,0,0,0,0.000000,0,0.000000,0,⋯,0,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.000000,0,0.000000
AL669831.5,0,0,0,0,0,0,0.000000,0,0.000000,0,⋯,0,0.000000,0.000000,4.286483,0.00000,0.000000,0,4.220721,0,0.000000
FAM87B,0,0,0,0,0,0,0.000000,0,0.000000,0,⋯,0,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.000000,0,0.000000
LINC00115,0,0,0,0,0,0,0.000000,0,0.000000,0,⋯,0,0.000000,4.040861,0.000000,5.81081,0.000000,0,4.906497,0,0.000000
FAM41C,0,0,0,0,0,0,0.000000,0,0.000000,0,⋯,0,0.000000,0.000000,0.000000,0.00000,0.000000,0,0.000000,0,0.000000
NOC2L,0,0,0,0,0,0,5.683571,0,5.199611,0,⋯,0,6.326213,0.000000,0.000000,0.00000,5.041668,0,4.220721,0,4.216369


In [40]:
ordered.genes<-sort(rownames(balf.samples$C100))

[1] "A1BG"     "A1BG-AS1" "A2M"      "A2M-AS1"  "A2ML1"    "A4GALT"

Finally, we apply a batch correction. The goal here is to account for sample-to-sample technical variability. 

At this point, we diverge from the Python preprocessing tutorial in order to leverage Seurat's built-in batch correction functions. To decrease run time, we will use reciprocal PCA instead of CCA. See https://satijalab.org/seurat/articles/integration_introduction.html and Seurat's other integration vignettes for additional details. To apply Combat as in scanpy, see commented code further below.

Note, the final input matrices to Tensor-cell2cell must be non-negative. We will demonstrate workarounds to negative counts in the tensor building tutorial.

In [107]:
batch.var <- 'Sample.ID' # the batch variable in the metadata

In [178]:
# get the HVGs for each sample separately
balf.samples <- lapply(balf.samples, 
                      function(so) FindVariableFeatures(so, selection.method = "vst", nfeatures = 2000))
                       
# find the common HVGs across samples
integration.features <- SelectIntegrationFeatures(object.list = balf.samples)

                       
# # to use CCA instead of reciprocal PCA, follow lines 10-12, instead of lines 14-22
# # find the integration anchors
# integration.anchors <- FindIntegrationAnchors(object.list = balf.samples, 
#                                               anchor.features = integration.features)    
                       
# calculate PCA on each sample separately
balf.samples <- lapply(X = balf.samples, FUN = function(x) {
    x <- ScaleData(x, features = integration.features, verbose = F)
    x <- RunPCA(x, features = integration.features, verbose = F)
})

# find the integration anchors
integration.anchors <- FindIntegrationAnchors(object.list = balf.samples, 
                                              anchor.features = integration.features, reduction = "rpca")

# do the batch correction
balf.corrected <- IntegrateData(anchorset = integration.anchors)

In [190]:
# 2000 top variable features were already calculated

# get PCA to 100 PCs
balf.corrected <- ScaleData(balf.corrected, verbose = F)
balf.corrected <- RunPCA(balf.corrected, npcs = 100, verbose = F)

In [193]:
balf.corrected

An object of class Seurat 
24900 features across 63102 samples within 2 assays 
Active assay: integrated (2000 features, 2000 variable features)
 1 other assay present: RNA
 1 dimensional reduction calculated: pca

Do the batch correction using Combat. This most closely emulated the steps taken in the scanpy tutorial, though there are still differences.

In [99]:
# # merge the balf samples

# # note, this is less stringent than scanpy's concat method and will retain all features
# balf.corrected<-merge(x = balf.samples[[1]], y = balf.samples[2:length(balf.samples)], 
#                       add.cell.ids = names(balf.samples), merge.data = TRUE)


# # prepare batch covariate
# batch<-balf.corrected@meta.data[[batch.var]]
# names(batch)<-row.names(balf.corrected@meta.data)
# batch<-as.factor(batch)

# # do the batch correction
# com <- ComBat(dat=balf.corrected@assays$RNA@data, batch=batch)

# # store the batch corrected matrix in the scale.data slot
# balf.corrected@assays$RNA@scale.data<-com

# # get the top 2000 highly variable genes
# balf.corrected<-FindVariableFeatures(balf.corrected, nfeatures=2000)

# # get PCA to 100 PCs, calculated on batch corrected matrix
# RunPCA(balf.corrected, features=VariableFeatures(balf.corrected), npcs=100, 
#        assay = 'RNA', slot = 'scale.data')

Finally, we can convert this batch-correct Seurat object to an AnnData object using SeuratDisk (see https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html for details). The resultant h5ad file should contain the same information as the saved h5ad file in the Python tutorial at the end of tutorial 01A.

In [196]:
out.path = '/data3/hratch/c2c_general/'
SaveH5Seurat(balf.corrected, filename = paste0(out.path, 'Rbatch_corrected_balf_covid.h5Seurat'))
Convert(paste0(out.path, 'Rbatch_corrected_balf_covid.h5Seurat'), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Adding data for integrated

Adding scale.data for integrated

Adding variable features for integrated

No feature-level metadata found for integrated

Adding cell embeddings for pca

Adding loadings for pca

No projected loadings for pca

Adding standard deviations for pca

No JackStraw data for pca

Validating h5Seurat file

Adding scale.data from integrated as X

Adding data from integrated as raw

Transfering meta.data to obs

Adding dimensional reduction information for pca

Adding feature loadings for pca



The .h5ad file, named below, can be read into scanpy using scanpy.read_h5ad(filename)

In [197]:
filename=paste0(out.path, 'Rbatch_corrected_balf_covid.h5ad')
print(filename)

[1] "/data3/hratch/c2c_general/Rbatch_corrected_balf_covid.h5ad"
